# Part 1

You reach the train's last stop and the closest you can get to your vacation island without getting wet. There aren't even any boats here, but nothing can stop you now: you build a raft. You just need a few days' worth of food for your journey.

You don't speak the local language, so you can't read any ingredients lists. However, sometimes, allergens are listed in a language you do understand. You should be able to use this information to determine which ingredient contains which allergen and work out which foods are safe to take with you on your trip.

You start by compiling a list of foods (your puzzle input), one food per line. Each line includes that food's ingredients list followed by some or all of the allergens the food contains.

Each allergen is found in exactly one ingredient. Each ingredient contains zero or one allergen. Allergens aren't always marked; when they're listed (as in (contains nuts, shellfish) after an ingredients list), the ingredient that contains each listed allergen will be somewhere in the corresponding ingredients list. However, even if an allergen isn't listed, the ingredient that contains that allergen could still be present: maybe they forgot to label it, or maybe it was labeled in a language you don't know.

For example, consider the following list of foods:
```
mxmxvkd kfcds sqjhc nhms (contains dairy, fish)
trh fvjkl sbzzf mxmxvkd (contains dairy)
sqjhc fvjkl (contains soy)
sqjhc mxmxvkd sbzzf (contains fish)
```

The first food in the list has four ingredients (written in a language you don't understand): mxmxvkd, kfcds, sqjhc, and nhms. While the food might contain other allergens, a few allergens the food definitely contains are listed afterward: dairy and fish.

The first step is to determine which ingredients can't possibly contain any of the allergens in any food in your list. In the above example, none of the ingredients kfcds, nhms, sbzzf, or trh can contain an allergen. Counting the number of times any of these ingredients appear in any ingredients list produces 5: they all appear once each except sbzzf, which appears twice.

Determine which ingredients cannot possibly contain any of the allergens in your list. How many times do any of those ingredients appear?

In [1]:
from collections import defaultdict


def get_input():
    with open('inputs/21') as f:
        return [parse_food(line) for line in f.read().splitlines()]
    

def parse_food(line):
    line = line.rstrip(')')
    parts = line.split(' (contains ')
    ings = parts[0].split(' ')
    allergens = [] if len(parts) == 1 else parts[1].split(', ')
    return {'ings': set(ings), 'allergens': set(allergens)}
    

def list_possible_ings_for_allergens(foods):
    possible_ings_for_allergens = {}
    for f in foods:
        for a in f['allergens']:
            if a not in possible_ings_for_allergens:
                possible_ings_for_allergens[a] = f['ings'].copy()
            else:
                possible_ings_for_allergens[a].intersection_update(f['ings'])
    return possible_ings_for_allergens


def list_all_ings_and_allergens(foods):
    all_ings = set()
    all_allergens = set()
    for f in foods:
        all_ings.update(f['ings'])
        all_allergens.update(f['allergens'])
    return {'ings': all_ings, 'allergens': all_allergens}


def find_safe_ings(foods):
    possible_ings_for_allergens = list_possible_ings_for_allergens(foods)
    dangerous_ings = set()
    for a, ings in possible_ings_for_allergens.items():
        dangerous_ings.update(ings)
    all_ings = list_all_ings_and_allergens(foods)['ings']
    return all_ings.difference(dangerous_ings)


def count_safe_ing_appearances(foods):
    safe_ings = find_safe_ings(foods)
    result = 0
    for f in foods:
        result += len(f['ings'].intersection(safe_ings))
    return result


# TESTS
test_file = '''mxmxvkd kfcds sqjhc nhms (contains dairy, fish)
trh fvjkl sbzzf mxmxvkd (contains dairy)
sqjhc fvjkl (contains soy)
sqjhc mxmxvkd sbzzf (contains fish)'''
test_inp = [parse_food(l) for l in test_file.splitlines()]
assert find_safe_ings(test_inp) == {'kfcds', 'nhms', 'sbzzf', 'trh'}
assert count_safe_ing_appearances(test_inp) == 5


def result1():
    foods = get_input()
    return count_safe_ing_appearances(foods)


result1()

2659

# Part 2

Now that you've isolated the inert ingredients, you should have enough information to figure out which ingredient contains which allergen.

In the above example:
```
mxmxvkd contains dairy.
sqjhc contains fish.
fvjkl contains soy.
```
Arrange the ingredients alphabetically by their allergen and separate them by commas to produce your canonical dangerous ingredient list. (There should not be any spaces in your canonical dangerous ingredient list.) In the above example, this would be mxmxvkd,sqjhc,fvjkl.

Time to stock your raft with supplies. What is your canonical dangerous ingredient list?

In [2]:
def match_allergens_with_ings(foods):
    unmatched_allergens = list_possible_ings_for_allergens(foods)
    result = {}
    while len(unmatched_allergens) > 0:
        allerg = min(unmatched_allergens, key= lambda a:len(unmatched_allergens[a]))
        ings = unmatched_allergens.pop(allerg)
        if len(ings) > 1:
            raise Exception('Need a more sophisticated algorithm.')
        if len(ings) == 0:
            raise Exception('There is no solution.')
        ing = list(ings)[0]
        result[allerg] = ing
        for a in unmatched_allergens:
            unmatched_allergens[a].difference_update(ings)
    return result


def result2():
    foods = get_input()
    matched = match_allergens_with_ings(foods)
    dangerous_ings = [matched[a] for a in sorted(matched)]
    return ','.join(dangerous_ings)


result2()

'rcqb,cltx,nrl,qjvvcvz,tsqpn,xhnk,tfqsb,zqzmzl'